In [98]:
import plotly.express as px
import pandas as pd
import numpy as np
import kaleido

IMPORTING ALL FILES FROM THE NAMES LISTED IN "mypart.csv"

In [99]:
file = "mypart.csv"
with open(file, 'r') as f2:
    #Cleaning the filename data
    filenames = f2.read().replace('\n', '').replace(' "', '').replace('"', '').replace(".txt", '').replace("gcn_Nissim", "gcn&Nissim")
    
path = 'task1/out_pdf/'
filenames = filenames.split(',')

filenames.reverse() #ORDER IS REVERSED AS I WAS ASKED TO SEARCH IN REVERSE ORDER (FROM THE BOTTOM)

In [100]:
#FUNCTION TO REMOVE THE FILES NOT IN R BAND
def r_band(light):
    for i in range(len(light)):
        if not light.band.values[i] in ('R','r', "R'", "r'"):
            light.band.values[i] = None
    light.dropna(inplace=True)

In [101]:
#FUNCTION TO REMOVE POINTS WITH 0 ERROR AND CORRUPTED DATA
def zero_error(light):
    for i in range(len(light)):
        if light.flux_err.values[i] == 0.0:
            light.flux_err.values[i] = None
        if type(light.index.values[i]) == str:
            light.flux_err.values[i] = None
    light.dropna(inplace=True)       

FUNCTION TO RETURN THE R, r, r' BAND DATA FROM A GIVEN FILE IN THE MAINSAMPLE FOLDER

In [102]:

def filter(f2):
     filename = "mainsample/"+f2
     try:
          light = pd.read_csv(filename+".txt", sep=r"\t+|\s+", engine="python", names= ["time_sec","flux","flux_err","band","source"],
         header=0, index_col=0)
          zero_error(light)
          r_band(light)
        
          #light.to_csv("task1/filtered/"+filename.replace('mainsample/','').replace("_flux","_filtered_flux")+".txt")
          return light

     except:
          light = pd.read_csv(filename+".txt", sep=r"\t+|\s+", engine="python", names= ["time_sec","flux","flux_err","band","source"],
         header=0, index_col=0)
          zero_error(light)
          r_band(light)
        
          #light.to_csv("task1/filtered/"+filename.replace('mainsample/','').replace("_flux","_filtered_flux")+".txt")
          return light


FUNCTION TO PLOT A GIVEN DATABASE AND STORE THE OUTPUT PLOT IN $task1/out$ _ $pdf$ FOLDER

In [103]:
def plot(light, f2):
    filename = f2
    if len(light) != 0:
        try:
            fig = px.scatter(
                        data_frame=light,
                        x=np.log10(light.index.values),
                        y=np.log10(light[1].values),
                        error_y= light[2].values / (light[1].values * np.log(10)),
                        color=light[3].values,
                        hover_data=[4],
                        title=filename,
                    )

            fig.update_layout(
                        xaxis_title='log Time (s)',
                        yaxis_title='log Flux (erg cm-2 s-1)'
                    )
            fig.write_image(path+f2+".pdf")
        #
        except:
            fig = px.scatter(
                data_frame=light,
                x=np.log10(light.index),
                y=np.log10(light['flux'].values),
                error_y= light['flux_err'].values / (light['flux'].values * np.log(10)),
                #color=light['band'].values,
                hover_data=['source'],
                title=filename,
            )
            fig.update_layout(
                xaxis_title='log Time (s)',
                yaxis_title='log Flux (erg cm-2 s-1)'
            )
        fig.show()
        fig.write_image(path+f2+".pdf")

In [104]:
#FUNCTION TO CONCATANATE N DATA FRAMES INTO A SINGLE DATAFRAME AND RETURN IT
def concat(*args):
    return pd.concat([x for x in args if not x.empty])

THE BELOW CELL INITIALIZES AN EMPTY DATABASE $all$ _ $light$

THEN IT APPENDS ALL THE DATA FROM EACH FILE TO $all$ _ $light$

In [105]:
i=0
all_light = pd.DataFrame()
for file in filenames:
    i+=1
    all_light = concat(all_light, filter(file))


In [106]:
all_light

,flux,flux_err,band,source
time_sec,,,,
179516.0,4.591653e-13,4.229069e-14,R,454
316028.0,1.386657e-13,2.554317e-14,R,454
373916.0,1.386657e-13,1.277158e-14,R,456
461480.0,5.942498e-14,3.283946e-15,R,458
354294.0,1.101461e-13,1.521724e-14,R,462
...,...,...,...,...
15497.0,1.163848e-13,5.359787e-15,r,Li_Liang
17006.0,1.042068e-13,4.798964e-15,r,Li_Liang
21430.0,8.509344e-14,4.702481e-15,r,Li_Liang


In [107]:
plot(all_light, "output")

/tmp/ipykernel_7158/1695642430.py:7: RuntimeWarning:

divide by zero encountered in log10

/home/darkwake/.local/lib/python3.10/site-packages/pandas/core/indexes/base.py:920: RuntimeWarning:

divide by zero encountered in log10



THE FOLLOWING CELL CALCULATES THE THRESHOLD VALUES THAT DEFINE BOUNDARIES OF OUTLIERS

In [108]:
flux_thres = (np.quantile(all_light.flux.values, 0.05) , np.quantile(all_light.flux.values, 0.95))
time_thres = (np.quantile(all_light.index.values, 0.05) , np.quantile(all_light.index.values, 0.95))

In [109]:
#FUNCTION TO FIND WHETHER A GIVEN VALUE IS IN BETWEEN TWO OTHER QUANTITIES
def between(a, min, max):
    if a >= min and a <= max:
        return True
    else: return False
#FUNCTION TO REMOVE THE OUTLIERS FROM A DATAFRAME BASED THE THRESHOLD BOUNDARIES
def rem_outliers(light, flux_thres, time_thres):
    for i in range(len(light)):
        if not (between(light.flux.values[i], flux_thres[0], flux_thres[1]) and between(light.index.values[i], time_thres[0], time_thres[1])):
            light.flux.values[i] = None

    light.dropna(inplace=True)

In [110]:
rem_outliers(all_light, flux_thres, time_thres)

PLOTTING THE OUTPUT AND STORING THE FINAL DATA BASE IN $outputf$ _ $converted$ _ $filtered$ _ $flux.txt$

In [111]:
all_light.to_csv("task1/filtered/outputf_converted_filtered_flux"+".txt")
plot(all_light, "outputf")